# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
import re
obj = {  
         'home_page': 'https://github.com/pypa/sampleproject',
         'keywords': 'sample setuptools development',
         'license': 'MIT'
      }


index_equal = len(max(obj.keys(), key=len))
for key, value in obj.items():
    print(key+ f'{{:>{index_equal-len(key)+2}}} '.format('=')+ f"'{value}'")

home_page = 'https://github.com/pypa/sampleproject'
keywords  = 'sample setuptools development'
license   = 'MIT'


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [2]:
import re
import pandas as pd

obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
# match = re.search("\d{2}-\d", obj[0])
df = pd.DataFrame(obj, columns = ['desc'])
df["group"] = df.desc.apply(lambda x: re.search("\d{2}-\d", x).group())
df

,desc,group
0,Евгения гр.ПМ19-1,19-1
1,Илья пм 20-4,20-4
2,Анна 20-3,20-3


3. Разбейте текст формулировки задачи 2 на слова.

In [3]:
import re
task='Написать регулярное выражение,которое позволит найти номера групп студентов.'
re.findall("\w+", task)

['Написать',
 'регулярное',
 'выражение',
 'которое',
 'позволит',
 'найти',
 'номера',
 'групп',
 'студентов']

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [4]:
import pandas as pd
import numpy as mp

In [5]:
recipes = pd.read_csv('recipes_sample.csv')
s = ['   id  ', ' minutes']
print(f'|{str(s[0])}|{str(s[1])} |')
print(f'|-----------------|')
for i in range(5):
  print(f'|{recipes["id"][i]: ^ 7}|{recipes["minutes"][i] : ^9}|')




|   id  | minutes |
|-----------------|
| 44123 |   90    |
| 67664 |   10    |
| 38798 |   30    |
| 35173 |   45    |
| 84797 |   25    |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [8]:
from bs4 import BeautifulSoup
import pandas as pd
recipes_sample = pd.read_csv("recipes_sample.csv")
with open("steps_sample.xml") as f:
    data=f.read()
steps_sample = BeautifulSoup(data, 'xml')

In [10]:
recipes_sample.loc[recipes_sample.id==170895]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
15754,leeks and parsnips sauteed or creamed,170895,27,8377,2006-05-31,21.0,this is good sauteed only or creamed. very eas...,9.0


In [ ]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

<IPython.core.display.Javascript object>

In [11]:
recipes=steps_sample.find_all('recipe')
def get_recipe_by_id(_id):
    current_recipe=recipes_sample.loc[recipes_sample.id==170895]
    for recipe in recipes:
        if recipe.find("id").text == str(_id):
            steps = recipe.find_all('step')
            l_step=0
            number = 1
            print(f"'{current_recipe.name.values[0]}'")
            for step in steps:
                print(f'{number}. '+step.text)
                number += 1
                l_step=max(l_step, len(step.text))
    print('-'*(l_step+3))
    print(f"Автор: {current_recipe.contributor_id.values[0]}")
    print(f"Среднее время приготовления: {current_recipe.minutes.values[0]} минут")
    
get_recipe_by_id(170895)

'leeks and parsnips  sauteed or creamed'
1. clean the leeks and discard the dark green portions
2. cut the leeks lengthwise then into one-inch pieces
3. melt the butter in a medium skillet , med
4. heat
5. add the garlic and fry 'til fragrant
6. add leeks and fry until the leeks are tender , about 6-minutes
7. meanwhile , peel and chunk the parsnips into one-inch pieces
8. place in a steaming basket and steam 'til they are as tender as you prefer
9. i like them fork-tender
10. drain parsnips and add to the skillet with the leeks
11. add salt and pepper
12. gently sautee together for 5-minutes
13. at this point you can serve it , or continue on and cream it:
14. in a jar with a screw top , add the half-n-half and arrowroot
15. shake 'til blended
16. turn heat to low under the leeks and parsnips
17. pour in the arrowroot mixture , stirring gently as you pour
18. if too thick , gradually add the water
19. let simmer for a couple of minutes
20. taste to adjust seasoning , probably an addit

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [13]:
import re
recipes=steps_sample.find_all('recipe')
def get_recipe_by_pattern(_id):
    pattern = '\d+\s(?:hour|hours|minute|minutes)'
    for recipe in recipes:
        if int(recipe.find("id").text) == _id:
            steps = recipe.find_all('step')
            for step in steps:
                time = re.findall(pattern, step.text)
                if time:
                    for interval in time:
                        print(interval)
get_recipe_by_pattern(25082)

20 minute
10 minute
2 hour
10 minute
20 minute
30 minute


In [14]:
recipes_sample.description.str

4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [16]:
import re
pattern='this[\w\s]*,(?:but| but)'
recipes_go_pattern = recipes_sample.loc[recipes_sample.description.str.match(pattern)==True]
print(f"Количество рецептов: {recipes_go_pattern.shape[0]}\n")
for recipe in range(3):
    print(f"Рецепт №{recipe}:\n    {recipes_go_pattern.description.values[recipe]}\n")

Количество рецептов: 134

Рецепт №0:
    this is a great meal eaten the same day ,but even better the next day , if you can wait! add your favourite spices, but try it first as it is and i think that you will enjoy the 'vegetable' taste. good for freezing.

Рецепт №1:
    this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers.

Рецепт №2:
    this is kind of similar to some of the other versions out there, but it is the best and easiest i have found



5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [17]:
import regex as re
recipes=steps_sample.find_all('recipe')
def get_recipe_by_pattern(_id):
    pattern = re.compile('\d+ \/ \d+') 
    for recipe in recipes:
        if int(recipe.find("id").text) == _id:
            steps = recipe.find_all('step')
            for step in steps:
                step=step.text
                a = re.search(pattern, step)
                if a:
                    t = a.group().split(' / ')
                    step = re.sub(pattern, f'{t[0]}/{t[1]}', step) 
                print(step)

get_recipe_by_pattern(72367)

mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [18]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [21]:
from bs4 import BeautifulSoup
import pandas as pd
recipes_sample = pd.read_csv("recipes_sample.csv")
with open("steps_sample.xml") as f:
    data=f.read()
steps_sample = BeautifulSoup(data, 'xml')

In [22]:
import nltk
recipes=steps_sample.find_all('recipe')
def get_words():
    words = set()
    for recipe in recipes:
        steps = recipe.find_all('step')
        for step in steps:
            step=step.text
            words.update(nltk.word_tokenize(step))
    real_words =set()
    for word in words:
        if word.isalpha():
            real_words.add(word)
    return real_words
print(len(get_words()))

14926


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [24]:
recipes=steps_sample.find_all('recipe')
recipes_sample = pd.read_csv("recipes_sample.csv")
[i for i in recipes_sample.id.values[:5]]

[44123, 67664, 38798, 35173, 84797]

In [25]:
import nltk

def get_sentences():
    ids = [i for i in recipes_sample.id.values[:5]]
    l_s = [0 for i in range(5)]
    
    for recipe in recipes:
        _id = recipe.find('id').text
        steps = recipe.find_all('step')
        l_sentences = 0
        for step in steps:
            step=step.text
            l_sentences+=len(nltk.sent_tokenize(step))
        for i in range(len(l_s)):
            if l_sentences>l_s[i]:
                ids[i] = int(_id)
                l_s[i] = l_sentences
                break
    return ids, l_s
ids, length_s = get_sentences()

In [26]:
print(ids)

[337926, 536360, 373670, 510490, 509506]


In [27]:
print(length_s)

[95, 73, 69, 66, 56]


In [28]:
recipes_sample.loc[recipes_sample.id.isin(ids)]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
13294,hamburger potpie with homemade crust,536360,160,1801884905,2018-07-17,73.0,"the flaky, buttery crust is only the beginning...",NaN
17562,meyer lemon marmalade,373670,150,1206943,2009-05-22,69.0,i only had a few meyer lemons from my first cr...,NaN
21733,pumpkin pie made from a real pumpkin,337926,195,1027170,2008-11-17,NaN,the best pumpkin pie you will ever taste!,NaN
25895,sticky cinnamon buns 2,510490,90,695933,2013-12-07,65.0,nothing beats sticky buns hot out of the oven!,NaN
28554,veal chuck chops with onions cabbage apples,509506,75,2958169,2013-11-15,NaN,the flavors for this scream out fall cooking! ...,NaN


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [29]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [30]:
import nltk

def get_words(this_id):
    for recipe in recipes:
        _id = int(recipe.find('id').text)
        if _id == this_id:
            step = recipe.find('step').text
            words = nltk.word_tokenize(step)
            parts = [b for a, b in nltk.pos_tag(words)]
            windows = [max(len(word), len(part)) for word, part in zip(words, parts)]
            print(*[f"{str(part):^{window}s}" for part, window in zip(parts, windows)])
            print(*[f"{str(word):^{window}s}" for word, window in zip(words, windows)])
get_words(241106)

TO  VB  DT     JJ       NN    :   VB    DT   JJ   CD     NNS     ,   RB    VBG   DT  NN  CC    VBG      RB    TO   VB     RB 
to make the balsamic marinade : combine the first 5  ingredients , slowly adding the oil and stirring briskly to combine well
